<a href="https://colab.research.google.com/github/marmurr/Deep-Learning/blob/main/Deep_Learning_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task:
* Implement automatic backpropagation (hint: topological sort, dfs)

* Implement those activations (2 of them) that you chose in lab 1 (RELu, Softplus)

* Implement gradient descent as a separate function

# Imports

In [131]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [143]:
import math
import numpy as np

class Parameter:
    def __init__(self, value: float, name: str, _children=()) -> None:
        self._value = value
        self._name = name

        self._grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)

    def __repr__(self) -> str:  # Representation (parameter - gradient)
        return f"Parameter {self._name} = {self._value}; dL/d[{self._name}] = {self._grad}"

    def __mul__(self, other):  # Multiplication of parameters
      other = other if isinstance(other, Parameter) else Parameter(other, "p")
      result = Parameter(
          self._value * other._value,
          f'{self._name} * {other._name}',
          _children=(self, other)
      )

      def _backward():
          self._grad += other._value * result._grad  # dL / dself
          other._grad += self._value * result._grad  # dL / dother

      result._backward = _backward

      return result

    def __pow__(self, other):  # Power of parameter
        out = Parameter(self._value ** other, f"{self._name} ** {other}")

        def _backward():
            self._grad += other * (self._value ** (other - 1)) * out._grad

        out._backward = _backward

        return out

    def __add__(self, other):  # Addition of parameters
        other = other if isinstance(other, Parameter) else Parameter(other, "p")
        result = Parameter(
            self._value + other._value,
            f'[{self._name} + {other._name}]',
            _children=(self, other)
        )

        def _backward():
            self._grad += 1.0 * result._grad  # dL / dself
            other._grad += 1.0 * result._grad  # dL / dother

        result._backward = _backward

        return result


    def sigmoid(self):  # Sigmoid activation function
        val = 1.0 / (1.0 + math.exp(-self._value))

        result = Parameter(
            val,
            f"σ({self._name})"
        )

        def _backward():
            self._grad += result._grad * val * (1 - val)

        result._backward = _backward

        return result

    def __neg__(self):  # Negation
        return self * (-1)

    def __sub__(self, other):  # Subtraction
        return self + (-other)

    def backward(self, grad=1.0):
        self._grad = grad
        topo_sort = []
        visited_nodes = set()

        def sort_topo(node):
            if node not in visited_nodes:
                visited_nodes.add(node)
                for child in node._prev:
                    sort_topo(child)
                topo_sort.append(node)
        sort_topo(self)

        for node in reversed(topo_sort):
            node._backward()

    def softplus(self):  # Softplus activation function
        val = np.log(1 + np.exp(self._value))
        result = Parameter(
            val,
            f"softplus({self._name})"
        )

        def _backward():
            self._grad += result._grad * (1.0 / (1.0 + np.exp(-self._value)))

        result._backward = _backward

        return result

    def ReLU(self):  # ReLU activation function
        val = np.maximum(0, self._value)
        result = Parameter(
            val,
            f"ReLU({self._name})"
        )

        def _backward():
            self._grad += result._grad * np.where(self._value > 0, 1, 0)

        result._backward = _backward

        return result

def gd(learning_rate: float, *parameters: Parameter) -> None:
    for parameter in parameters:
        parameter._value -= learning_rate * parameter._grad
        parameter._grad = 0

# Backward method test

In [133]:
a = Parameter(3.0, 'a')
b = Parameter(2.0, 'b')
c = Parameter(5.0, 'c')
m = Parameter(5.0, 'd')
u = a * b
v = u + c
L = v * m
L.backward()
print(L)
print(v)
print(u)
print(m)
print(c)
print(b)
print(a)

Parameter [a * b + c] * d = 55.0; dL/d[[a * b + c] * d] = 0
Parameter [a * b + c] = 11.0; dL/d[[a * b + c]] = 0
Parameter a * b = 6.0; dL/d[a * b] = 0
Parameter d = 5.0; dL/d[d] = 0
Parameter c = 5.0; dL/d[c] = 0
Parameter b = 2.0; dL/d[b] = 0
Parameter a = 3.0; dL/d[a] = 0


In [134]:
x1 = Parameter(3.0, 'x1')
x2 = Parameter(4.0, 'x2')

w1 = Parameter(1.0, 'w1')
w2 = Parameter(2.0, 'w2')

x1w1 = x1 * w1
x2w2 = x2 * w2
xw = x1w1 + x2w2
out = xw.ReLU()
out.backward()
print(out._grad)
print(xw._grad)
print(x2w2._grad)
print(x1w1._grad)
print(x1._grad)
print(w1._grad)
print(x2._grad)
print(w2._grad)

0
1.0
0.0
0.0
0.0
0.0
0.0
0.0


In [135]:
x1 = Parameter(3.0, 'x1')
x2 = Parameter(4.0, 'x2')

w1 = Parameter(1.0, 'w1')
w2 = Parameter(2.0, 'w2')

x1w1 = x1 * w1
x2w2 = x2 * w2
xw = x1w1 + x2w2
out = xw.softplus()
out.backward()
print(out._grad)
print(xw._grad)
print(x2w2._grad)
print(x1w1._grad)
print(x1._grad)
print(w1._grad)
print(x2._grad)
print(w2._grad)

0
0.999983298578152
0.0
0.0
0.0
0.0
0.0
0.0


# Gradient descent test

In [144]:
W = Parameter(0.2, 'W')
b = Parameter(0.5, 'b')
x = Parameter(1, 'x')
learning_rate = 0.001
n_epochs = 10
target = Parameter(0.75, 't')
for n in range(n_epochs):
    y = (W * x + b).ReLU()
    loss = (y - target) ** 2
    loss.backward()
    gd(learning_rate, W, b)
    print(f"loss after {n} epochs :{loss}")

loss after 0 epochs :Parameter [ReLU([W * x + b]) + t * p] ** 2 = 0.0025000000000000044; dL/d[[ReLU([W * x + b]) + t * p] ** 2] = 1.0
loss after 1 epochs :Parameter [ReLU([W * x + b]) + t * p] ** 2 = 0.0025000000000000044; dL/d[[ReLU([W * x + b]) + t * p] ** 2] = 1.0
loss after 2 epochs :Parameter [ReLU([W * x + b]) + t * p] ** 2 = 0.0025000000000000044; dL/d[[ReLU([W * x + b]) + t * p] ** 2] = 1.0
loss after 3 epochs :Parameter [ReLU([W * x + b]) + t * p] ** 2 = 0.0025000000000000044; dL/d[[ReLU([W * x + b]) + t * p] ** 2] = 1.0
loss after 4 epochs :Parameter [ReLU([W * x + b]) + t * p] ** 2 = 0.0025000000000000044; dL/d[[ReLU([W * x + b]) + t * p] ** 2] = 1.0
loss after 5 epochs :Parameter [ReLU([W * x + b]) + t * p] ** 2 = 0.0025000000000000044; dL/d[[ReLU([W * x + b]) + t * p] ** 2] = 1.0
loss after 6 epochs :Parameter [ReLU([W * x + b]) + t * p] ** 2 = 0.0025000000000000044; dL/d[[ReLU([W * x + b]) + t * p] ** 2] = 1.0
loss after 7 epochs :Parameter [ReLU([W * x + b]) + t * p] ** 

In [145]:
print((W * x + b).ReLU())

Parameter ReLU([W * x + b]) = 0.7; dL/d[ReLU([W * x + b])] = 0.0
